# Setup

In [1]:
!git clone https://github.com/onlookertozip/SamsungElectronics_Gumi.git

Cloning into 'SamsungElectronics_Gumi'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 77 (delta 23), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (77/77), 26.55 MiB | 7.69 MiB/s, done.
Resolving deltas: 100% (23/23), done.


# Part1: Langchain 기초


## 1. 환경 구성

### 1) 라이브러리 설치

In [1]:
!pip install -q langchain==0.0.181 langchain_openai==0.1.21 httpx==0.27.2 tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 11.2 MB/s eta 0:00:00
Reason for being yanked: Regression. AzureChatOpenAI json-mode broken. Fixed in 0.1.22.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.6/934.6 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.20 requ

### 2) OpenAI 인증키 설정

In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## 2. LLM Chain

### 1) Prompt + LLM


In [3]:
from langchain_openai import ChatOpenAI

# model
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# chain 실행
result = llm.invoke("지구의 자전 주기는?")

In [4]:
result.content

'지구의 자전 주기는 약 24시간입니다. 이는 하루 동안 지구가 자전하는 시간을 나타내며, 이에 따라 낮과 밤이 생기게 됩니다.'

In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
prompt

ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are an expert in astronomy. Answer the question. <Question>: {input}'))])

In [6]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# chain 연결 (LCEL)
chain = prompt | llm

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

AIMessage(content='지구의 자전 주기는 약 24시간입니다. 이것은 하루의 길이를 의미하며, 지구가 자전하면서 자전축 주위를 한 바퀴 도는데 약 24시간이 걸리기 때문입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 30, 'total_tokens': 105, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c9746477-e9f8-4095-8298-c45ff30cb7bc-0', usage_metadata={'input_tokens': 30, 'output_tokens': 75, 'total_tokens': 105})

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser
prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
output_parser = StrOutputParser()

# LCEL chaining
chain = prompt | llm | output_parser

# chain 호출
chain.invoke({"input": "지구의 자전 주기는?"})

'지구의 자전 주기는 약 24시간 혹은 정확히 23시간 56분 4.1초입니다. 이것은 지구가 자전하는 데 걸리는 시간으로, 하루의 길이를 결정짓는 요소 중 하나입니다.'

### 2) Multiple Chains

In [8]:
prompt1 = ChatPromptTemplate.from_template("translates {korean_word} to English.")
prompt2 = ChatPromptTemplate.from_template(
    "explain {english_word} using oxford dictionary to me in Korean."
)

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain1 = prompt1 | llm | StrOutputParser()

chain1.invoke({"korean_word":"미래"})

'The word "미래" translates to "future" in English.'

In [9]:
chain2 = (
    {"english_word": chain1}
    | prompt2
    | llm
    | StrOutputParser()
)

chain2.invoke({"korean_word":"미래"})

'미래는 어떤 시간으로부터 보아 예상되는 또는 예견되는 일이나 상황을 가리키는 말이다.'

## 3. Prompt
  

### 1) PromptTemplate

In [10]:
from langchain_core.prompts import PromptTemplate

# 'name'과 'age'라는 두 개의 변수를 사용하는 프롬프트 템플릿을 정의
template_text = "안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(name="홍길동", age=30)

filled_prompt

'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.'

In [11]:
# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n아버지를 아버지라 부를 수 없습니다.")
              + "\n\n{language}로 번역해주세요."
)

combined_prompt

PromptTemplate(input_variables=['age', 'language', 'name'], template='안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n{language}로 번역해주세요.')

In [12]:
combined_prompt.format(name="홍길동", age=30, language="영어")

'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n영어로 번역해주세요.'

In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
chain.invoke({"age":30, "language":"영어", "name":"홍길동"})

'Hello, my name is Hong Gil-dong and I am 30 years old.\n\nI cannot call my father "father".'

### 2) ChatPromptTemplate

In [14]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

messages = chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")
messages

[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.'),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?')]

In [15]:
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

'태양계에서 가장 큰 행성은 목성입니다. 목성은 질량, 부피, 지름 등 여러 측면에서 가장 큰 행성으로 알려져 있습니다.'

### 3) Message

In [16]:
# MessagePromptTemplate 활용

from langchain_core.prompts import SystemMessagePromptTemplate,  HumanMessagePromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("이 시스템은 천문학 질문에 답변할 수 있습니다."),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)


messages = chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")
messages

[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.'),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?')]

In [17]:
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

'태양계에서 가장 큰 행성은 목성입니다. 목성은 지름이 약 142,984km로 태양계에서 가장 큰 행성이며, 매우 큰 가스 행성입니다.'

## 5. Model Parameter


### 1) 모델 클래스 유형

#### LLM

In [18]:
from langchain_openai import OpenAI

llm = OpenAI()

llm.invoke("한국의 대표적인 관광지 3군데를 추천해주세요.")

'\n\n1. 경복궁\n2. 남산 타워\n3. 제주도'

#### ChatModel

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 여행 전문가입니다."),
    ("user", "{user_input}"),
])

chain = chat_prompt | chat
chain.invoke({"user_input": "안녕하세요? 한국의 대표적인 관광지 3군데를 추천해주세요."})

AIMessage(content='안녕하세요! 한국의 대표적인 관광지로는 다음 3군데를 추천해드립니다:\n\n1. 경복궁 (Gyeongbokgung): 서울에 위치한 경복궁은 조선 시대의 궁궐로, 한국의 역사와 문화를 경험할 수 있는 곳입니다. 경복궁 근처에는 국립고궁박물관과 청와대도 있어서 함께 방문하기 좋습니다.\n\n2. 부산 해운대 해변 (Haeundae Beach): 부산의 대표적인 해변으로 유명한 해운대 해변은 아름다운 바다 풍경과 다양한 레스토랑, 상점이 있는 곳입니다. 여름에는 많은 관광객들이 모여 바다에서 즐거운 시간을 보낼 수 있습니다.\n\n3. 경주 (Gyeongju): 경주는 한국의 역사적인 도시로, 많은 문화유산과 유적지가 있습니다. 세계문화유산으로 지정된 석굴암이나 첨성대, 불국사 등을 방문하여 한국의 고대 역사를 체험할 수 있습니다.\n\n이 외에도 한국에는 다양한 멋진 관광지가 많이 있으니, 방문하고 싶은 지역이 있으시면 더 자세한 정보를 알려주시면 더 많은 도움을 드릴 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 441, 'prompt_tokens': 59, 'total_tokens': 500, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-59b4ae34-caff-440

### 2) 모델 파라미터 설정


#### 모델에 직접 파라미터를 전달 (모델 생성 시점)

In [20]:
from langchain_openai import ChatOpenAI

# 모델 파라미터 설정
params = {
    "temperature": 0.7,         # 생성된 텍스트의 다양성 조정
    "max_tokens": 100,          # 생성할 최대 토큰 수
    "frequency_penalty": 0.5,   # 이미 등장한 단어의 재등장 확률
    "presence_penalty": 0.5,    # 새로운 단어의 도입을 장려
    "stop": ["\n"]              # 정지 시퀀스 설정
}


# 모델 인스턴스를 생성할 때 설정, model_kwargs 제거 후 params에 직접 전달
model = ChatOpenAI(model="gpt-3.5-turbo-0125", **params)


# 모델 호출
question = "태양계에서 가장 큰 행성은 무엇인가요?"
response = model.invoke(input=question)

# 전체 응답 출력
print(response)

content='태양계에서 가장 큰 행성은 목성(Jupiter)입니다. 목성은 태양계 내에서 질량과 부피가 가장 크며, 지름이 또한 가장 큰 행성입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 29, 'total_tokens': 96, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-db00e633-0fce-43ab-9421-deaf26101900-0' usage_metadata={'input_tokens': 29, 'output_tokens': 67, 'total_tokens': 96}


#### 모델에 직접 파라미터를 전달 (모델 호출 시점)

In [21]:
# 모델 파라미터 설정
params = {
    "temperature": 0.7,         # 생성된 텍스트의 다양성 조정
    "max_tokens": 10,          # 생성할 최대 토큰 수
}

# 모델 인스턴스를 호출할 때 전달
response = model.invoke(input=question, **params)

# 문자열 출력
print(response.content)

태양계에서 가장 큰


#### 모델에 추가적인 파라미터를 전달
- bind 메서드를 사용

In [22]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

model = ChatOpenAI(model="gpt-3.5-turbo-0125", max_tokens=100)

messages = prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")

before_answer = model.invoke(messages)

# # binding 이전 출력
print(before_answer)

# 모델 호출 시 추가적인 인수를 전달하기 위해 bind 메서드 사용 (응답의 최대 길이를 10 토큰으로 제한)
chain = prompt | model.bind(max_tokens=10)

after_answer = chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

# binding 이후 출력
print(after_answer)

content='태양계에서 가장 큰 행성은 목성입니다. 목성은 지름이 약 142,984km로 가장 큰 행성이며, 대기 중 99% 이상이 수소와 헬륨으로 이루어져 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 58, 'total_tokens': 129, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-05ab287d-a58c-47eb-b3ff-a4bd33067916-0' usage_metadata={'input_tokens': 58, 'output_tokens': 71, 'total_tokens': 129}
content='태양계에서 가장 큰' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 58, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'au

## 6. Output Parsers


### 1) CSV Parser

In [23]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [24]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

chain = prompt | llm | output_parser

chain.invoke({"subject": "popular Korean cusine"})

['Bibimbap', 'Kimchi', 'Bulgogi', 'Japchae', 'Tteokbokki']

### 2) JSON Parser

In [25]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

# 자료구조 정의 (pydantic)
class CusineRecipe(BaseModel):
    name: str = Field(description="name of a cusine")
    recipe: str = Field(description="recipe to cook the cusine")

In [26]:
# 출력 파서 정의
output_parser = JsonOutputParser(pydantic_object=CusineRecipe)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe to cook the cusine", "type": "string"}}, "required": ["name", "recipe"]}
```


In [27]:
# prompt 구성
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)

print(prompt)

input_variables=['query'] partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe to cook the cusine", "type": "string"}}, "required": ["name", "recipe"]}\n```'} template='Answer the user query.\n{format_instructions}\n{query}\n'


In [28]:
chain = prompt | model | output_parser

chain.invoke({"query": "Let me know how to cook Bibimbap"})

{'name': 'Bibimbap',
 'recipe': 'Bibimbap is a traditional Korean dish that consists of mixed rice with vegetables, meat, and a spicy sauce. To cook Bibimbap, you will need the following ingredients: cooked rice, assorted vegetables (such as spinach, bean sprouts, carrots, zucchini, mushrooms), protein (such as beef, chicken, tofu), Gochujang sauce (Korean chili paste), soy sauce, sesame oil'}